### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [1]:
import numpy as np
import pandas as pd
my_list_first =pd.read_html("https://fbref.com/en/comps/9/2021-2022/")
my_list = my_list_first.copy()

In [2]:
squad_std_stats = my_list[2]
cols = squad_std_stats.columns
index = np.array(squad_std_stats['Unnamed: 0_level_0', 'Squad'])
squad_std_stats.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
               Squad               # Pl                Age               Poss   
0            Arsenal                 27               24.4               52.8   
1        Aston Villa                 31               26.1               46.5   
2          Brentford                 29               25.4               44.8   
3           Brighton                 26               26.2               54.4   
4            Burnley                 23               28.5               40.2   

  Playing Time                    Performance      ... Per 90 Minutes  \
            MP Starts   Min   90s         Gls Ast  ...         G+A-PK   
0           38    418  3420  38.0          60  41  ...           2.53   
1           38    418  3420  38.0          50  42  ...           2.34   
2           38    418  3420  38.0          46  33  ...           1.92   
3           38    418  3420  38.0          40  30  ...           1.74   
4           38    418  3420  38.0          32  26  ...           1.50   

  Expected                      Per 90 Minutes                              
        xG  npxG   xAG npxG+xAG             xG   xAG xG+xAG  npxG npxG+xAG  
0     67.3  62.3  46.4    108.7           1.77  1.22   2.99  1.64     2.86  
1     47.9  45.9  37.9     83.8           1.26  1.00   2.26  1.21     2.21  
2     51.1  46.4  36.6     83.0           1.35  0.96   2.31  1.22     2.18  
3     53.0  47.5  37.3     84.9           1.40  0.98   2.38  1.25     2.23  
4     43.8  42.8  34.1     76.8           1.15  0.90   2.05  1.13     2.02  

[5 rows x 29 columns]

In [3]:
# df_std = squad_std_stats[unnamed_cols].droplevel(0, axis=1)
squad_std_stats.index = index
df_pl_time = squad_std_stats["Playing Time"]
df_pl_time.head()

,MP,Starts,Min,90s
Arsenal,38,418,3420,38.0
Aston Villa,38,418,3420,38.0
Brentford,38,418,3420,38.0
Brighton,38,418,3420,38.0
Burnley,38,418,3420,38.0


In [4]:
from library import edit_standard_stats_table

In [5]:
df_list = edit_standard_stats_table(my_list[2], my_list[3])

In [6]:
col_0 = [["Performance", "Penalty Kicks"], [ "Goals", "Expected", "Launched", "Passes", "Goal Kicks", "Crosses", "Sweeper"]]


In [7]:
gk_df = my_list[4]
index = np.array(gk_df["Unnamed: 0_level_0", "Squad"])
gk_df.index = index
gk_df = gk_df[col_0[0]]
gk_df = gk_df.droplevel(0, axis=1)
gk_df.drop(columns=["W", "D", "L"], inplace=True)

In [8]:
from library import clean_opp_df
gk_opp_df = my_list[5].copy()
gk_opp_df = clean_opp_df(gk_opp_df)
gk_opp_df.index = index
gk_opp_df = gk_opp_df[col_0[0]]
gk_opp_df = gk_opp_df.droplevel(0, axis=1)
gk_opp_df.drop(columns=["W_Opp", "D_Opp", "L_Opp"], inplace=True)
gk_overall = gk_df.join(gk_opp_df)

In [9]:
gk_overall.columns

Index(['GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'CS', 'CS%', 'PKatt', 'PKA',
       'PKsv', 'PKm', 'Save%', 'GA_Opp', 'GA90_Opp', 'SoTA_Opp', 'Saves_Opp',
       'Save%_Opp', 'CS_Opp', 'CS%_Opp', 'PKatt_Opp', 'PKA_Opp', 'PKsv_Opp',
       'PKm_Opp', 'Save%_Opp'],
      dtype='object')

In [10]:
df = my_list[8].copy()
df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Standard            \
               Squad               # Pl                90s      Gls   Sh  SoT   
0            Arsenal                 27               38.0       60  581  186   
1        Aston Villa                 31               38.0       50  461  159   
2          Brentford                 29               38.0       46  436  141   
3           Brighton                 26               38.0       40  482  141   
4            Burnley                 23               38.0       32  405  119   

                                                     Expected                \
   SoT%  Sh/90 SoT/90  G/Sh G/SoT  Dist  FK PK PKatt       xG  npxG npxG/Sh   
0  32.0  15.29   4.89  0.09  0.30  17.8  22  5     8     67.3  62.3    0.11   
1  34.5  12.13   4.18  0.10  0.30  17.6  20  3     3     47.9  45.9    0.10   
2  32.3  11.47   3.71  0.09  0.28  16.3  11  6     6     51.1  46.4    0.11   
3  29.3  12.68   3.71  0.07  0.26  17.7  15  4     7     53.0  47.5    0.10   
4  29.4  10.66   3.13  0.08  0.26  17.2  14  1     2     43.8  42.8    0.11   

                 
   G-xG np:G-xG  
0  -7.3    -7.3  
1   2.1     1.1  
2  -5.1    -6.4  
3 -13.0   -11.5  
4 -11.8   -11.8

In [11]:
df_opp = clean_opp_df(my_list[9].copy())
df_opp.head()

Standard                                                        \
             Gls_Opp Sh_Opp SoT_Opp SoT%_Opp Sh/90_Opp SoT/90_Opp G/Sh_Opp   
Arsenal           47    419     142     33.9     11.03       3.74     0.10   
Aston Villa       52    471     144     30.6     12.39       3.79     0.10   
Brentford         54    498     174     34.9     13.11       4.58     0.10   
Brighton          44    472     143     30.3     12.42       3.76     0.09   
Burnley           51    598     173     28.9     15.74       4.55     0.08   

                                                       Expected           \
            G/SoT_Opp Dist_Opp FK_Opp PK_Opp PKatt_Opp   xG_Opp npxG_Opp   
Arsenal          0.30     17.6     16      5         6     51.2     46.5   
Aston Villa      0.33     17.4     22      4         6     54.4     49.6   
Brentford        0.28     18.2     22      5         5     54.0     50.0   
Brighton         0.29     18.6     21      3         4     49.8     46.9   
Burnley          0.27     17.7     16      5         5     63.8     59.9   

                                              
            npxG/Sh_Opp G-xG_Opp np:G-xG_Opp  
Arsenal            0.11     -4.2        -4.5  
Aston Villa        0.11     -2.4        -1.6  
Brentford          0.10      0.0        -1.0  
Brighton           0.10     -5.8        -5.9  
Burnley            0.10    -12.8       -13.9

#### 1. Fix the df_opp func
#### 2.Write test cases 

In [22]:
from library import clean_main_df
clean_main_df(df).head()

Standard                                                         \
                 Gls   Sh  SoT  SoT%  Sh/90 SoT/90  G/Sh G/SoT  Dist  FK PK   
Arsenal           60  581  186  32.0  15.29   4.89  0.09  0.30  17.8  22  5   
Aston Villa       50  461  159  34.5  12.13   4.18  0.10  0.30  17.6  20  3   
Brentford         46  436  141  32.3  11.47   3.71  0.09  0.28  16.3  11  6   
Brighton          40  482  141  29.3  12.68   3.71  0.07  0.26  17.7  15  4   
Burnley           32  405  119  29.4  10.66   3.13  0.08  0.26  17.2  14  1   

                  Expected                              
            PKatt       xG  npxG npxG/Sh  G-xG np:G-xG  
Arsenal         8     67.3  62.3    0.11  -7.3    -7.3  
Aston Villa     3     47.9  45.9    0.10   2.1     1.1  
Brentford       6     51.1  46.4    0.11  -5.1    -6.4  
Brighton        7     53.0  47.5    0.10 -13.0   -11.5  
Burnley         2     43.8  42.8    0.11 -11.8   -11.8

In [13]:
shooting_df = clean_main_df(my_list[8].copy()).join(clean_opp_df(my_list[9].copy()))

In [14]:
from library import edit_gk_talbes

df1 = edit_gk_talbes(my_list[4].copy(), my_list[5].copy())
df1.head()

,SoTA,Saves,Save%,CS,CS%,PKsv,Save%,SoTA_Opp,Saves_Opp,Save%_Opp,CS_Opp,CS%_Opp,PKsv_Opp,Save%_Opp
Arsenal,147,100,70.7,13,34.2,0,0.0,194.0,131.0,71.1,11.0,28.9,3.0,37.5
Aston Villa,149,96,66.4,11,28.9,1,20.0,162.0,112.0,69.8,8.0,21.1,0.0,0.0
Brentford,179,125,71.5,9,23.7,0,0.0,147.0,101.0,71.4,12.0,31.6,0.0,0.0
Brighton,146,102,71.9,11,28.9,0,0.0,146.0,105.0,74.0,13.0,34.2,1.0,20.0
Burnley,178,127,73.0,9,23.7,0,0.0,120.0,88.0,72.5,16.0,42.1,0.0,0.0


In [17]:
from library import merge_dfs
df2 = merge_dfs(my_list[4].copy(), my_list[5].copy())
df2.head()

Playing Time                    Performance                   \
                      MP Starts   Min   90s          GA  GA90 SoTA Saves   
Arsenal               38     38  3420  38.0          48  1.26  147   100   
Aston Villa           38     38  3420  38.0          54  1.42  149    96   
Brentford             38     38  3420  38.0          56  1.47  179   125   
Brighton              38     38  3419  38.0          44  1.16  146   102   
Burnley               38     38  3420  38.0          53  1.39  178   127   

                       ...                                  Penalty Kicks  \
            Save%   W  ... W_Opp D_Opp L_Opp CS_Opp CS%_Opp     PKatt_Opp   
Arsenal      70.7  22  ...  13.0   3.0  22.0   11.0    28.9           8.0   
Aston Villa  66.4  13  ...  19.0   6.0  13.0    8.0    21.1           3.0   
Brentford    71.5  13  ...  18.0   7.0  13.0   12.0    31.6           6.0   
Brighton     71.9  12  ...  11.0  15.0  12.0   13.0    34.2           7.0   
Burnley      73.0   7  ...  17.0  14.0   7.0   16.0    42.1           2.0   

                                                
            PKA_Opp PKsv_Opp PKm_Opp Save%_Opp  
Arsenal         5.0      3.0     0.0      37.5  
Aston Villa     3.0      0.0     0.0       0.0  
Brentford       6.0      0.0     0.0       0.0  
Brighton        4.0      1.0     2.0      20.0  
Burnley         1.0      0.0     1.0       0.0  

[5 rows x 38 columns]

In [15]:
df1.columns

Index(['SoTA', 'Saves', 'Save%', 'CS', 'CS%', 'PKsv', 'Save%', 'SoTA_Opp',
       'Saves_Opp', 'Save%_Opp', 'CS_Opp', 'CS%_Opp', 'PKsv_Opp', 'Save%_Opp'],
      dtype='object')

In [18]:
df2.columns

MultiIndex([( 'Playing Time',         'MP'),
            ( 'Playing Time',     'Starts'),
            ( 'Playing Time',        'Min'),
            ( 'Playing Time',        '90s'),
            (  'Performance',         'GA'),
            (  'Performance',       'GA90'),
            (  'Performance',       'SoTA'),
            (  'Performance',      'Saves'),
            (  'Performance',      'Save%'),
            (  'Performance',          'W'),
            (  'Performance',          'D'),
            (  'Performance',          'L'),
            (  'Performance',         'CS'),
            (  'Performance',        'CS%'),
            ('Penalty Kicks',      'PKatt'),
            ('Penalty Kicks',        'PKA'),
            ('Penalty Kicks',       'PKsv'),
            ('Penalty Kicks',        'PKm'),
            ('Penalty Kicks',      'Save%'),
            ( 'Playing Time',     'MP_Opp'),
            ( 'Playing Time', 'Starts_Opp'),
            ( 'Playing Time',    'Min_Opp'),
          